In [28]:
import pandas as pd
import plotly.express as px
df = pd.read_csv("dataset/jp_lca_dat.csv", delimiter=',', encoding='latin-1')
print(df.head())
print(df.dtypes)

   Product id         Data S2 Name        Group  id      Product_details  \
0           1  Wheat & Rye (Bread)  starch_rich   1          Wheat bread   
1           1  Wheat & Rye (Bread)  starch_rich   4          Wheat bread   
2           1  Wheat & Rye (Bread)  starch_rich   5  Wheat and rye bread   
3           1  Wheat & Rye (Bread)  starch_rich   6          Wheat bread   
4           1  Wheat & Rye (Bread)  starch_rich   7          Wheat bread   

     Country Weight  Land Use (m2*year)  Arable Land Use (m2*year)  \
0  Australia   0.9%                7.78                       7.78   
1     France   1.4%                5.42                       5.42   
2     France   1.4%               29.28                      29.28   
3     France   0.2%                1.44                       1.44   
4    Germany   1.1%                1.31                       1.31   

   Pasture Land Use (m2*year)  Biodiversity (sp*yr*10^14)  \
0                         0.0                      220.91   


In [29]:
meat_products = ['Beef', 'Mutton', 'Pork', 'Chicken', 'Duck', 'Turkey']
fish_products = [
    'Grass carp, silver carp, big head carp, common carp, catfish',
    'Tilapia, African catfish', 'Rainbow trout', 'Trout', 'Sea bass, sea bream', 'Salmon',
    'Common carp, tench, roach, perch, sander, pike', 'Tambaqui', 'Pangasius',
    'Giant river prawn', 'Tiger prawn', 'Amazon river prawn', 'Giant giver prawn', 'Shrimp'
]
dairy_products = ["Cow's milk", "Cow's cheese (Gouda)"] 

In [30]:
def categorize_product(product):
    if product in meat_products:
        return 'Meat'
    elif product in fish_products:
        return 'Fish'
    elif product in dairy_products:
        return 'Vegetarian'  # Dairy products are part of vegetarian but not vegan
    else:
        return 'Vegan'  # Default to vegan if not in any of the above categories

# Now apply this function to your DataFrame
df['diet_group'] = df['Product_details'].apply(categorize_product)

# Adjust the vegan group to include dairy for the vegetarian group
df['diet_group'] = df.apply(lambda x: 'Vegetarian' if x['diet_group'] == 'Vegan' and x['Product_details'] in dairy_products else x['diet_group'], axis=1)

In [31]:
# Assume df is your DataFrame and it contains 'Country', 'GHG Emissions', and 'diet_group' columns
# Aggregate GHG emissions by country and diet group
agg_data = df.groupby(['Country', 'diet_group'])['GHG Emissions (kg CO2eq, IPCC2013 incl CC feedbacks)'].mean().reset_index()

# Create the choropleth map
fig = px.choropleth(
    agg_data,
    locations="Country",
    locationmode='country names',  # Make sure your country data matches the locationmode
    color="GHG Emissions (kg CO2eq, IPCC2013 incl CC feedbacks)",
    hover_name="Country",
    hover_data={"diet_group": True, "GHG Emissions (kg CO2eq, IPCC2013 incl CC feedbacks)": ':.2f'},
    title="GHG Emissions by Diet Group and Country",
    labels={"GHG Emissions (kg CO2eq, IPCC2013 incl CC feedbacks)": "GHG Emissions (kg CO2eq)"},
    category_orders={"diet_group": ["Vegan", "Vegetarian", "Fish", "Meat"]},  # Optional: to ensure consistent color mapping
    color_continuous_scale=px.colors.sequential.Plasma
)

fig.show()

In [32]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assuming agg_data is prepped with necessary 'Country', 'mean_emissions', and 'diet_group'
fig = make_subplots(rows=1, cols=2, subplot_titles=["GHG Emissions", "Diet Group"])

# Facet by diet group
fig = px.choropleth(
    agg_data,
    locations="Country",
    locationmode='country names',
    color="GHG Emissions (kg CO2eq, IPCC2013 incl CC feedbacks)",
    facet_col="diet_group",  # Create a map for each diet group
    facet_col_wrap=2,  # Wrap at 2 columns
    color_continuous_scale='Viridis'
)

fig.update_layout(title_text='GHG Emissions by Diet Group and Country')
fig.show()

In [33]:
# Map for GHG Emissions
fig_ghg = px.choropleth(
    agg_data,
    locations="Country",
    locationmode='country names',
    color="GHG Emissions (kg CO2eq, IPCC2013 incl CC feedbacks)",
    color_continuous_scale='Reds',
    title="Global GHG Emissions by Country",
    showlegend=False
)
fig_ghg.show()

# Map for Diet Groups
fig_diet = px.choropleth(
    agg_data,
    locations="Country",
    locationmode='country names',
    color="diet_group",
    title="Diet Group Distribution by Country",
    color_discrete_map={"Vegan": "green", "Vegetarian": "yellow", "Fish": "blue", "Meat": "red"}
)
fig_diet.show()

In [35]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Create a Dash application
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

# Assume fig_ghg and fig_diet are already created as described previously
# Now let's define the layout of your Dash application
app.layout = html.Div(children=[
    html.H1(children='Environmental Impact Dashboard', style={'textAlign': 'center'}),

    html.Div(children=[
        html.Div([
            dcc.Graph(
                id='ghg-emissions-map',
                figure=fig_ghg
            )], className="six columns"),

        html.Div([
            dcc.Graph(
                id='diet-groups-map',
                figure=fig_diet
            )], className="six columns"),
    ], className="row"),

    html.Footer(children=[
        html.P('Data Source: Provided by User', style={'textAlign': 'center'})
    ], style={'marginTop': 50})
], style={'maxWidth': '1200px', 'margin': '0 auto'})

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)
